In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import pyLDAvis.gensim

In [12]:
data = pd.read_excel('Lucky_May15.xlsx')

In [13]:
text = data['2504Please describe any problems you had.'].dropna().tolist()
text = [str(review[3:-2]) for review in text]

In [14]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

def text_clean(text):
    raw = text.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    stemmed_tokens = [i for i in stemmed_tokens if len(i) > 2]
    return stemmed_tokens
    

In [15]:
texts = [text_clean(review) for review in text]

In [16]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

In [17]:
# convert tokenized documents into a document-term matrix
# %time  
corpus = [dictionary.doc2bow(text) for text in texts]

In [30]:
# generate LDA model
# %time 
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=50, random_state=np.random.seed(4141))

In [31]:
followers_data =  pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(followers_data)